In [5]:
from ..model_triton import *

seq_len = 32
vocab_size = 1000 #12000
layers = 1 #12
emb_dim= 512
nheads = 2 #8
ffn_dim = 256
params = init_transformer_gpt2(vocab_size, emb_dim, layers, nheads, ffn_dim, seq_len)

# Print params if needed
# for p_grp_ind, p_grp in enumerate(params):
#     for p_ind, p in enumerate(p_grp):
#         print(p_grp_ind, p_ind, p.shape)

ImportError: attempted relative import with no known parent package

In [1]:
# TEST 1 test_tlayer_attn_head_fwd -> works fine
attn_head_params = params[2][2][1]
torch.manual_seed(0)
q = torch.randn((seq_len, emb_dim), dtype=torch.float32, device="cuda")
k = torch.randn((seq_len, emb_dim), dtype=torch.float32, device="cuda")
v = torch.randn((seq_len, emb_dim), dtype=torch.float32, device="cuda")
y_mask = torch.tril(torch.ones((seq_len, seq_len), dtype=torch.bool, device="cuda"))
#y_mask = torch.unsqueeze(y_mask,0)

res1=tlayer_attn_head_fwd(attn_head_params, (q,k,v), y_mask, False)

batched_qkv = tuple([torch.unsqueeze(it,0) for it in (q,k,v)])
batched_y_mask = torch.unsqueeze(y_mask,0)
res2=t_tlayer_attn_head_fwd(attn_head_params, batched_qkv, batched_y_mask, False)[0]
torch.allclose(res1, res2, 1e-04)

NameError: name 'params' is not defined

In [114]:
# TEST 2 test_tlayer_attn_heads_fwd -> numerical differences due to vmap being used in one of them
attn_heads_params = params[2][2]
torch.manual_seed(0)
q = torch.randn((seq_len, emb_dim), dtype=torch.float32, device="cuda")
k = torch.randn((seq_len, emb_dim), dtype=torch.float32, device="cuda")
v = torch.randn((seq_len, emb_dim), dtype=torch.float32, device="cuda")
y_mask = torch.tril(torch.ones((seq_len, seq_len), dtype=torch.bool, device="cuda"))

res1=tlayer_attn_heads_fwd(attn_heads_params[:1], (q,k,v), y_mask, False)

batched_qkv = tuple([torch.unsqueeze(it,0) for it in (q,k,v)])
batched_y_mask = torch.unsqueeze(y_mask,0)
res2=t_tlayer_attn_heads_fwd(attn_heads_params, batched_qkv, batched_y_mask, False)[0]
torch.allclose(res1, res2, 1e-01)

False

In [115]:
# TEST 3 test_tlayer_attn_fwd -> numerical differences, due to vmap being used
attn_fwd_params = params[2][2:-6]
torch.manual_seed(0)
q = torch.randn((seq_len, emb_dim), dtype=torch.float32, device="cuda")
k = torch.randn((seq_len, emb_dim), dtype=torch.float32, device="cuda")
v = torch.randn((seq_len, emb_dim), dtype=torch.float32, device="cuda")
y_mask = torch.tril(torch.ones((seq_len, seq_len), dtype=torch.bool, device="cuda"))
#y_mask = torch.unsqueeze(y_mask,0)

res1=tlayer_attn_fwd(attn_fwd_params, (q,k,v), y_mask, False)

batched_qkv = tuple([torch.unsqueeze(it,0) for it in (q,k,v)])
batched_y_mask = torch.unsqueeze(y_mask,0)
res2=t_tlayer_attn_fwd(attn_fwd_params, batched_qkv, batched_y_mask, False)[0]
torch.allclose(res1, res2, 1e-01)

False